In [1]:
import numpy as np
import pandas as pd
from numpy import random

import re

import random

import warnings
warnings.filterwarnings('ignore')

seed = 1998

In [4]:
# 데이터 불러오는 부분

area = 'Gangnam' # 데이터 불러오는 지역
data = pd.read_csv(f'../../crawings/user_csv/{area}/user_concat.csv')
rest = pd.read_csv(f'../../crawings/area_csv/{area}/rest_concat_{area}.csv')

data['userid'] = data['link'].apply(lambda x: str(re.findall(r'my/[a-z0-9]+/', x))[5:-3])
data = data[['userid', 'rest']]
data = data.drop_duplicates()
random.seed(seed)

In [7]:
user2idx = {user:idx for idx, user in enumerate(data['userid'].unique())}
idx2user = {idx:user for idx, user in enumerate(data['userid'].unique())}

## 패딩 때문에 0 비워놓음
item2idx = {item:(idx+1) for idx, item in enumerate(data['rest'].unique())}
idx2item = {(idx+1):item for idx, item in enumerate(data['rest'].unique())}

data['user_code'] = data['userid'].map(user2idx)
data['rest_code'] = data['rest'].map(item2idx)
rest['rest_code'] = rest['url'].map(item2idx)

In [14]:
data['rand'] = data['rest'].apply(lambda x : random.random())
_user = data['userid'].value_counts().reset_index()
_user.columns = ['userid', 'cnt']
data = pd.merge(data, _user, how = 'left', on = 'userid')
data = data[data['cnt'] > 5].reset_index(drop = True)
data = data[~(data['userid'] == '')].reset_index(drop = True)
data = data.sort_values(['userid', 'rand']).reset_index(drop = True)
data['tem'] = 1
data['seq'] = data.groupby('userid')['tem'].apply(lambda x : x.cumsum())
data['cnt'] = data['cnt'] // 5 + 1

In [15]:
# train/ test 분할하는 부분
train = data[data['tem'] + 1 < data['seq']]
test = data[data['tem'] + 1 >= data['seq']]

In [18]:
user = train.groupby('user_code')['rest_code'].unique().to_frame().reset_index()
user['user'] = user['user_code'].map(idx2user)
rest = rest.drop(['review', 'len','rating','count'], axis = 1)

,user_code,rest_code,user
0,0,"[2062, 2840, 875, 2841, 2867, 2855, 2846, 1, 2...",6130db4973adbe125329a3e4
1,4,"[2433, 2391, 2719, 1228, 265, 2646, 1048, 2440...",5dafbac38f87a842bcb2ed5e
2,12,"[832, 1245, 1061, 1428]",5e251c708f87a842bcd7b4ce
3,13,"[505, 490, 1, 293, 147, 2125, 1226]",5cf8f9149fa0794fb6921317
4,22,"[1561, 1798, 318, 1602, 19, 2062, 600, 1557, 9...",5d9b10208f87a842bca49a9a
...,...,...,...
32677,396071,"[2841, 2870, 2838, 2876]",5ff3db65a95a075d8049a8b1
32678,396072,"[2838, 2841, 2855, 2870]",5cdd4ef5f3c1899b7e2c1be1
32679,396170,"[2846, 2849, 2868, 2863]",6107d8f05a49a18e3be5bea2
32680,396378,"[2841, 2854, 2849, 2863]",5bf35aabe1ca0bee57c838df


In [26]:
train[['userid', 'rest', 'user_code', 'rest_code']].to_csv('G_train2.csv', index = False)
test[['userid', 'rest', 'user_code', 'rest_code']].to_csv('G_test2.csv', index = False)

user.to_csv('user.csv', index = False)
rest.to_csv('rest.csv', index = False)